In [4]:
import pandas
import numpy
import os


In [9]:
cur_dir = os.getcwd()
dataset_dir = cur_dir + '/head'
ext = '_head'

In [16]:
users = pandas.read_csv(dataset_dir + '/user' + ext + '.csv')
tips = pandas.read_csv(dataset_dir + '/tip' + ext + '.csv')
reviews = pandas.read_csv(dataset_dir + '/review' + ext + '.csv')
checkins = pandas.read_csv(dataset_dir + '/checkin' + ext + '.csv')
businesses = pandas.read_csv(dataset_dir + '/business' + ext + '.csv')

In [31]:
def get_user_active_weeks_in_city(users, reviews):
    """ 
    Given a set of users and a set of reviews This function determines the percentage of active weeks that a user
    has placed a review in any given city (from the set of all cities which have a review)
    """
    
    # collect all cities in the dataset
    # For each user get all reviews associated with that user
    # Divide those reviews into buckets based off of weeks and cities
    # calculate percentage of weeks in each city
    pass


def get_all_cities(businesses):
    """ Returns a list of all the cities in a dataset """
    cities = set(businesses['city'])
    return cities

def cluster_cities(businesses):
    

In [32]:
get_all_cities(businesses)

{'Ajax',
 'Avondale',
 'Beachwood',
 'Chandler',
 'Charlotte',
 'Cleveland',
 'Edinburgh',
 'Gilbert',
 'Las Vegas',
 'Madison',
 'Matthews',
 'Mc Kees Rocks',
 'McMurray',
 'Mesa',
 'Middleton',
 'Mississauga',
 'Montr\xc3\xa9al',
 'North York',
 'Parma Heights',
 'Phoenix',
 'Pickering',
 'Pineville',
 'Pittsburgh',
 'Richmond Heights',
 'Richmond Hill',
 'Rocky River',
 'Scottsdale',
 'Stanley',
 'Toronto'}

In [30]:
b = businesses.keys()
nonA = [ba for ba in b if 'Attributes' not in ba and 'attributes' not in ba and 'hours' not in ba]
nonAdf = businesses[nonA]
print(nonA)
# print(nonAdf)


['postal_code', 'city', 'stars', 'latitude', 'business_id', 'categories', 'name', 'longitude', 'is_open', 'neighborhood', 'review_count', 'state', 'address']


In [19]:
reviews.merge(businesses, on='business_id')

,funny,user_id,review_id,text,business_id,stars_x,date,useful,cool,hours.Wednesday,...,attributes.Caters,attributes.AcceptsInsurance,attributes.RestaurantsReservations,attributes.Music.video,attributes.Ambience.romantic,attributes.Music.jukebox,attributes.Ambience.upscale,attributes.RestaurantsTakeOut,attributes.BikeParking,attributes.OutdoorSeating
